In [ ]:
%%capture
%run 2-visualization.ipynb

In [ ]:
from tslearn.clustering import TimeSeriesKMeans, silhouette_score
from sklearn.preprocessing import MinMaxScaler

The first thing to do when doing clustering is to normalize or standardize the data.

The reason is that we want everything on the same scale in order to avoid scale differences to be included in the distance evaluation.

Here we chose to normalize with the minmax scaler.

In [ ]:
# let's normalize the numerical features of the dataframe
for df in datasets:
    df[numerics] = MinMaxScaler().fit_transform(df[numerics])

In [ ]:
dataframes['modena']

TimeSeriesKmeans requires a 3D array, so let's create it from all the datasets

Given the high dimensionality of the data, we decided to use only the POLLUTANTS as variables to do clustering. The choice was made by visualizing the data and realising that the first variables had almost always groups overlapping with each other. This means that those variables have low clustering power. This saves some computing time because we reduce the complexity of the problem.

In [ ]:
# Prepare input data
X = []

for province in provinces:
    group_data = dataframes[province][pollutants].values
    
    # Reshape to (n_samples, n_timestamps, n_features)
    group_data = np.expand_dims(group_data, axis=0)
    X.append(group_data)

# Stack the list of arrays to create a 3D array
X = np.vstack(X)

In [ ]:
X

When applying KMeans, we have to specify the number of clusters. This implies that we have to find the optimal number among the possible ones.
To do this, we examine the inertia and the silhouette score. They are both measures of how well separated the clusters are, therefore they can be used as goodness of fit measures.

The lower the inertia, the more the series are grouped towards the cluster center.

The higher the silhouette, the more the series are close to the other ones in the same cluster and far from the ones in other clusters.

In [ ]:
inertia = []
silhouette = []
K = list(range(2, 9))

for k in K:
    km = TimeSeriesKMeans(n_clusters=k, n_init=5, metric='dtw', random_state=0)
    
    labels = km.fit_predict(X)
    
    silhouette_avg = silhouette_score(X, labels, metric='dtw')
    
    inertia.append(km.inertia_)
    silhouette.append(silhouette_avg)

While the graph for inertia might be not clear in determining the otpimal numbre of clusters, slihouette score graph clearly shows how 4 is the optimal number

In [ ]:
# Plotting the results
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(K, inertia, marker='o')
plt.title('Inertia vs. Number of Clusters')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')

plt.subplot(1, 2, 2)
plt.plot(K, silhouette, marker='o')
plt.title('Silhouette Score vs. Number of Clusters')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')

plt.tight_layout()
plt.show()

In [ ]:
km4 = TimeSeriesKMeans(n_clusters=4, n_init=5, metric='dtw', random_state=0)

In [ ]:
clusters = km4.fit_predict(X)

In [ ]:
list(zip(provinces, clusters))

In [ ]:
full_df = pre_plots(clusters, 'cluster')

In [10]:
full_df

,date,TG,TN,TX,HU,PP,QQ,RR,CO,NH3,NMVOC,NO2,NO,O3,PANS,PM10,PM2.5,SO2,province,cluster
0,2018-01-01,0.205145,0.205367,0.251791,0.548308,0.711119,0.161726,0.628560,0.420886,0.216774,0.562685,0.617173,0.256141,0.167308,0.176811,0.161572,0.232706,0.347452,bologna,2
1,2018-01-02,0.229347,0.233994,0.267284,0.613952,0.678239,0.122727,0.372361,0.323778,0.187606,0.417026,0.520601,0.266229,0.167517,0.115745,0.076993,0.129220,0.285891,bologna,2
2,2018-01-03,0.251359,0.260581,0.280632,0.673535,0.668511,0.093520,0.235223,0.273497,0.172013,0.366878,0.476585,0.297929,0.163380,0.076908,0.047600,0.084460,0.245343,bologna,2
3,2018-01-04,0.271179,0.285126,0.291836,0.727055,0.674235,0.072817,0.180085,0.257319,0.164546,0.379776,0.467699,0.341759,0.156577,0.057909,0.059120,0.085131,0.224448,bologna,2
4,2018-01-05,0.288808,0.307630,0.300897,0.774513,0.689083,0.059393,0.176985,0.264445,0.161151,0.429278,0.479822,0.390031,0.148546,0.055914,0.098880,0.118955,0.221110,bologna,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9832,2020-12-24,0.351234,0.394720,0.325084,0.699873,0.762849,0.054686,0.371546,0.338842,0.117776,0.176182,0.286269,0.088219,0.189914,0.063350,0.261670,0.293133,0.235777,rimini,0
9833,2020-12-25,0.326647,0.361871,0.308206,0.637598,0.738300,0.057967,0.449940,0.253435,0.099408,0.101470,0.196627,0.029621,0.229135,0.054169,0.179326,0.167317,0.226687,rimini,0
9834,2020-12-26,0.291914,0.310997,0.286264,0.583675,0.709917,0.055718,0.542109,0.214478,0.099868,0.065422,0.136896,0.000000,0.275081,0.055359,0.122429,0.087289,0.232692,rimini,0
9835,2020-12-27,0.248513,0.241494,0.261063,0.555265,0.679248,0.043307,0.653370,0.243739,0.125956,0.078560,0.121357,0.011779,0.330150,0.068673,0.111089,0.085254,0.251988,rimini,0


In [14]:
plots('tskmeans', clusters, 'cluster', full_df[pollutants + ['date', 'cluster']])